# Python Sample Code for Optimizing Study Schedule for Students using Hybrid Particle Swarm Optimization and Simulated Annealing (PSO-SA) Approach
Caine Ivan R. Bautista (2022-0378)

## Import Libraries

In [1]:
import random
import numpy as np
from datetime import datetime, timedelta
import copy
import math

## Global Parameters for PSO and SA

In [2]:
min_temp=0 
max_temp=2000
cooling_rate=0.05

inertia = 0.73
personal = 1.494
social = 1.494
particles_size=5
max_iter=10

## Particle Object

In [3]:
class Particle:
    def __init__(self, num_subjects, study_time):
        self.num_subjects = num_subjects
        self.study_time = study_time
        self.schedules = {subject: random.uniform(0, self.study_time / self.num_subjects) for subject in range(self.num_subjects)}
        self.best_schedules = self.schedules
        self.best_fitness = float('inf')
        self.velocity = {subject: 0 for subject in range(self.num_subjects)}

    # Normalize schedule with minimum time allocation
    def normalize_schedule(self):
        # Handle 0 time
        if sum(self.schedules.values()) == 0:
            return {subject: self.study_time / self.num_subjects for subject in self.schedules}
        
        for subject in self.schedules:
            self.schedules[subject] = self.schedules[subject]

        total_time = sum(self.schedules.values())
        scale_factor = self.study_time / total_time

        # Scale the new schedules to match the current available time
        for subject in self.schedules:
            self.schedules[subject] *= scale_factor

## Proposed Algorithm in Python

In [4]:
class PSOSA:
    def __init__(self, num_subjects, study_time, priorities, breaktimes, particles_size=particles_size, max_iter=max_iter):
        self.num_subjects = num_subjects
        self.study_time = study_time  # Maximum study time per day (in hours)
        self.priorities = priorities  # Priority for each subject
        self.breaktimes = breaktimes  # Breaks for each subject (in hours)
        self.particles_size = particles_size  # Particle swarm size
        self.max_iter = max_iter  # Maximum number of iterations
        self.inertia = inertia
        self.personal = personal
        self.social = social

        self.particles = [Particle(self.num_subjects, self.study_time) for _ in range(self.particles_size)]
        self.best_particle = copy.deepcopy(self.particles[0])

    def fitness_function(self, schedule):
        total_time = sum(schedule.values())
        if total_time == 0:
            return float('inf')
    
        # Imbalance penalty: penalize uneven allocation of study time across subjects
        # Calculate Gini coefficient for time imbalance
        gini_numerator = 0
        for sched_i in schedule.values():
            for sched_j in schedule.values():
                gini_numerator += abs(sched_i - sched_j)
        gini_denominator = 2 * (len(schedule)**2) * (total_time / len(schedule))
        gini_coefficient = gini_numerator / gini_denominator
            
        penalty_weight_imbalance = 10  # Weight for imbalance penalty
        imbalance_penalty = penalty_weight_imbalance * (gini_coefficient**2)

        # Break penalty: penalize excessive or inefficient breaks
        break_penalty = 0
        for subject, break_time in self.breaktimes.items():
            # Assume an optimal break ratio (e.g., 1:5 for breaks to study time)
            optimal_break_ratio = 0.2
            allocated_break_time = schedule[subject] * optimal_break_ratio
            break_penalty += (break_time - allocated_break_time) ** 2  # Penalize deviation

        penalty_weight_break = 5  # Weight for break penalty
        break_penalty = penalty_weight_break * break_penalty

        # Gap penalty: penalize gaps between subjects
        gap_penalty = 0
        allocated_times = sorted(schedule.values(), reverse=True)
        for i in range(len(allocated_times) - 1):
            gap_penalty += (allocated_times[i] - allocated_times[i + 1]) ** 2  # Penalize large gaps

        penalty_weight_gap = 3  # Weight for gap penalty
        gap_penalty = penalty_weight_gap * gap_penalty

        # Fitness calculation
        fitness = 0
        for subject, time in schedule.items():
            fitness += self.priorities[subject] * time

        # Total penalty
        total_penalty = imbalance_penalty + gap_penalty + break_penalty

        # Return the negative fitness with penalties
        return -(fitness - total_penalty)

    def update_particles(self):
        for index, particle in enumerate(self.particles):
            for subject in particle.schedules:
                # Update velocity using the PSO formula
                diversification = self.inertia * particle.velocity[subject]
                personal_influence = self.personal * random.uniform(0, 1) * (particle.best_schedules[subject] - particle.schedules[subject])
                social_influence = self.social * random.uniform(0, 1) * (self.best_particle.best_schedules[subject] - particle.schedules[subject])

                particle.velocity[subject] = diversification + personal_influence + social_influence

                # Update the schedule using the velocity, ensuring no negative study time
                particle.schedules[subject] = max(0, particle.schedules[subject] + particle.velocity[subject])

            # Normalize the schedule to fit within study time constraints
            particle.normalize_schedule()

    def simulated_annealing(self, min_temp=min_temp, max_temp=max_temp, cooling_rate=cooling_rate):
        curr_temp = max_temp

        while curr_temp > min_temp:
            # Create a neighbor by copying the best particle
            neighbor = copy.deepcopy(self.best_particle)
            r_subject = random.choice(list(neighbor.schedules.keys()))

            # Generate a valid random change for the selected schedule subject
            neighbor.schedules[r_subject] = random.uniform(0, self.study_time)

            # Normalize the schedule
            neighbor.normalize_schedule()

            # Calculate fitness for the neighbor
            neighbor_fitness = self.fitness_function(neighbor.schedules)

            # Apply the simulated annealing acceptance condition
            if neighbor_fitness < self.best_particle.best_fitness or random.random() < math.exp((self.best_particle.best_fitness - neighbor_fitness) / curr_temp):
                self.best_particle.best_schedules = neighbor.schedules
                self.best_particle.best_fitness = neighbor_fitness

            # Reduce the temperature
            curr_temp *= cooling_rate

    def run(self):
        for iteration in range(self.max_iter):
            #print(f"Iteration {iteration + 1}")

            # Update the personal best for each particle
            for index, particle in enumerate(self.particles):
                schedule_fitness = self.fitness_function(particle.schedules)
                if schedule_fitness < particle.best_fitness:
                    particle.best_schedules = copy.deepcopy(particle.schedules)
                    particle.best_fitness = schedule_fitness

                #print(f"Particle {index}: Best Schedule {particle.best_schedules} | Fitness {particle.best_fitness}")

            # Update the global best particle
            self.set_best()

            # Perform simulated annealing to refine the global best particle
            self.simulated_annealing()

            # Update particles' velocities and positions
            self.update_particles()

        return self.best_particle

    def set_best(self):
        for particle in self.particles:
            if particle.best_fitness < self.best_particle.best_fitness:
                self.best_particle = copy.deepcopy(particle)

## Helper Function for Formatting the Generated Schedule

In [5]:
# Function to generate time frames with study and break time in chunks
def format(result, start_time, breaktimes):
    current_time = datetime.strptime(start_time, "%I:%M %p")  # Parse start time
    
    for subject, duration in result.best_schedules.items():
        if duration >= 0:
            # Start with study time in chunks
            remaining_study_time = duration
            print(f"Subject {subject + 1}:")
            while remaining_study_time > 0:
                break_minutes = int(breaktimes[subject] * 60)  # Break time in minutes
                study_time = min(remaining_study_time, break_minutes*5/60)  # Set max chunk based on 1:5 ratio
                end_time = current_time + timedelta(hours=study_time)
                session_duration_minutes = int(study_time * 60)  # Convert duration to minutes
                

                print(f"\t{current_time.strftime("%I:%M %p")} - {end_time.strftime("%I:%M %p")} "
                    f"({session_duration_minutes} minutes) (Break: {break_minutes} minutes)")
                
                # Update current time after study time and break
                current_time = end_time + timedelta(minutes=break_minutes)
                remaining_study_time -= study_time  # Decrease remaining study time     

## Sample Input

### Student 1

In [6]:
# Example parameters for the study schedule
num_subjects = 5  # Number of subjects
study_time = 10 # Total study time per day in hours (m hours)
priorities = {0: 3, 1: 1, 2: 4, 3: 5, 4: 2}  # Priority for each subject
breaktimes = {n:10/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "10:00 PM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 2.007814323173546, 1: 1.4663141761548026, 2: 2.295315481153164, 3: 2.5559516717044843, 4: 1.6746043478140036}
Schedule Fitness:  -30.241019298241433
Subject 1:
	10:00 PM - 10:50 PM (50 minutes) (Break: 10 minutes)
	11:00 PM - 11:50 PM (50 minutes) (Break: 10 minutes)
	12:00 AM - 12:20 AM (20 minutes) (Break: 10 minutes)
Subject 2:
	12:30 AM - 01:20 AM (50 minutes) (Break: 10 minutes)
	01:30 AM - 02:08 AM (37 minutes) (Break: 10 minutes)
Subject 3:
	02:18 AM - 03:08 AM (50 minutes) (Break: 10 minutes)
	03:18 AM - 04:08 AM (50 minutes) (Break: 10 minutes)
	04:18 AM - 04:56 AM (37 minutes) (Break: 10 minutes)
Subject 4:
	05:06 AM - 05:56 AM (50 minutes) (Break: 10 minutes)
	06:06 AM - 06:56 AM (50 minutes) (Break: 10 minutes)
	07:06 AM - 07:56 AM (50 minutes) (Break: 10 minutes)
	08:06 AM - 08:09 AM (3 minutes) (Break: 10 minutes)
Subject 5:
	08:19 AM - 09:09 AM (50 minutes) (Break: 10 minutes)
	09:19 AM - 10:09 AM (50 minutes) (Break: 10 minutes)
	10:19 AM

### Student 2

In [7]:
# Example parameters for the study schedule
num_subjects = 6  # Number of subjects
study_time = 6 # Total study time per day in hours (m hours)
priorities = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6}  # Priority for each subject
breaktimes = {n:20/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "8:00 AM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 0.26396070452409476, 1: 0.5035612087783197, 2: 0.8407757256733813, 3: 1.1715586040006438, 4: 1.519263420244487, 5: 1.7008803367790728}
Schedule Fitness:  -23.263020990370464
Subject 1:
	08:00 AM - 08:15 AM (15 minutes) (Break: 20 minutes)
Subject 2:
	08:35 AM - 09:06 AM (30 minutes) (Break: 20 minutes)
Subject 3:
	09:26 AM - 10:16 AM (50 minutes) (Break: 20 minutes)
Subject 4:
	10:36 AM - 11:46 AM (70 minutes) (Break: 20 minutes)
Subject 5:
	12:06 PM - 01:37 PM (91 minutes) (Break: 20 minutes)
Subject 6:
	01:57 PM - 03:37 PM (100 minutes) (Break: 20 minutes)
	03:57 PM - 03:59 PM (2 minutes) (Break: 20 minutes)
Day 2:
Generated Schedule:  {0: 0.12617376663890154, 1: 0.37308910539917545, 2: 0.7574345550956207, 3: 1.1605288887425766, 4: 1.634014550232514, 5: 1.9487591338912116}
Schedule Fitness:  -23.15947773024719
Subject 1:
	08:00 AM - 08:07 AM (7 minutes) (Break: 20 minutes)
Subject 2:
	08:27 AM - 08:49 AM (22 minutes) (Break: 20 minutes)
Subject 3:
	09:

### Student 3

In [8]:
# Example parameters for the study schedule
num_subjects = 4  # Number of subjects
study_time = 6 # Total study time per day in hours (m hours)
priorities = {0: 4, 1: 2, 2: 1, 3: 3}  # Priority for each subject
breaktimes = {n:20/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "6:00 AM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 1.8134997207601107, 1: 1.3332089012283679, 2: 1.2095596919276876, 3: 1.6437316860838342}
Schedule Fitness:  -15.492921598461441
Subject 1:
	06:00 AM - 07:40 AM (100 minutes) (Break: 20 minutes)
	08:00 AM - 08:08 AM (8 minutes) (Break: 20 minutes)
Subject 2:
	08:28 AM - 09:48 AM (79 minutes) (Break: 20 minutes)
Subject 3:
	10:08 AM - 11:21 AM (72 minutes) (Break: 20 minutes)
Subject 4:
	11:41 AM - 01:20 PM (98 minutes) (Break: 20 minutes)
Day 2:
Generated Schedule:  {0: 1.8221162179070671, 1: 1.3558577537711638, 2: 1.1963414814567324, 3: 1.625684546865037}
Schedule Fitness:  -15.514293875937454
Subject 1:
	06:00 AM - 07:40 AM (100 minutes) (Break: 20 minutes)
	08:00 AM - 08:09 AM (9 minutes) (Break: 20 minutes)
Subject 2:
	08:29 AM - 09:50 AM (81 minutes) (Break: 20 minutes)
Subject 3:
	10:10 AM - 11:22 AM (71 minutes) (Break: 20 minutes)
Subject 4:
	11:42 AM - 01:20 PM (97 minutes) (Break: 20 minutes)
Day 3:
Generated Schedule:  {0: 1.8314371185746916, 1

### Student 4

In [9]:
# Example parameters for the study schedule
num_subjects = 7  # Number of subjects
study_time = 9 # Total study time per day in hours (m hours)
priorities = {0: 6, 1: 2, 2: 4, 3: 3, 4: 1, 5: 5, 6: 7}  # Priority for each subject
breaktimes = {n:12/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "8:00 AM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 2.121692668466354, 1: 0.5170589547635329, 2: 1.1852730723910168, 3: 0.8678636979647986, 4: 0.20830464064365578, 5: 1.7126471932274845, 6: 2.3871597725431584}
Schedule Fitness:  -42.03466085594555
Subject 1:
	08:00 AM - 09:00 AM (60 minutes) (Break: 12 minutes)
	09:12 AM - 10:12 AM (60 minutes) (Break: 12 minutes)
	10:24 AM - 10:31 AM (7 minutes) (Break: 12 minutes)
Subject 2:
	10:43 AM - 11:14 AM (31 minutes) (Break: 12 minutes)
Subject 3:
	11:26 AM - 12:26 PM (60 minutes) (Break: 12 minutes)
	12:38 PM - 12:49 PM (11 minutes) (Break: 12 minutes)
Subject 4:
	01:01 PM - 01:53 PM (52 minutes) (Break: 12 minutes)
Subject 5:
	02:05 PM - 02:18 PM (12 minutes) (Break: 12 minutes)
Subject 6:
	02:30 PM - 03:30 PM (60 minutes) (Break: 12 minutes)
	03:42 PM - 04:24 PM (42 minutes) (Break: 12 minutes)
Subject 7:
	04:36 PM - 05:36 PM (60 minutes) (Break: 12 minutes)
	05:48 PM - 06:48 PM (60 minutes) (Break: 12 minutes)
	07:00 PM - 07:24 PM (23 minutes) (Break: 12 min

### Student 5

In [10]:
# Example parameters for the study schedule
num_subjects = 3  # Number of subjects
study_time = 5 # Total study time per day in hours (m hours)
priorities = {0: 2, 1: 3, 2: 1}  # Priority for each subject
breaktimes = {n:10/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "12:00 AM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 1.6845178514902297, 1: 1.8020435073828696, 2: 1.5134386411269007}
Schedule Fitness:  -9.719464591885197
Subject 1:
	12:00 AM - 12:50 AM (50 minutes) (Break: 10 minutes)
	01:00 AM - 01:50 AM (50 minutes) (Break: 10 minutes)
	02:00 AM - 02:01 AM (1 minutes) (Break: 10 minutes)
Subject 2:
	02:11 AM - 03:01 AM (50 minutes) (Break: 10 minutes)
	03:11 AM - 04:01 AM (50 minutes) (Break: 10 minutes)
	04:11 AM - 04:19 AM (8 minutes) (Break: 10 minutes)
Subject 3:
	04:29 AM - 05:19 AM (50 minutes) (Break: 10 minutes)
	05:29 AM - 06:10 AM (40 minutes) (Break: 10 minutes)
Day 2:
Generated Schedule:  {0: 1.669421823221413, 1: 1.781999314077109, 2: 1.5485788627014776}
Schedule Fitness:  -9.71978661780102
Subject 1:
	12:00 AM - 12:50 AM (50 minutes) (Break: 10 minutes)
	01:00 AM - 01:50 AM (50 minutes) (Break: 10 minutes)
	02:00 AM - 02:00 AM (0 minutes) (Break: 10 minutes)
Subject 2:
	02:10 AM - 03:00 AM (50 minutes) (Break: 10 minutes)
	03:10 AM - 04:00 AM (50 minute

### Student 6

In [11]:
# Example parameters for the study schedule
num_subjects = 6  # Number of subjects
study_time = 9 # Total study time per day in hours (m hours)
priorities = {0: 5, 1: 3, 2: 6, 3: 3, 4: 2, 5: 1}  # Priority for each subject
breaktimes = {n:8/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "8:00 AM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 2.112874284817991, 1: 1.6775356721368617, 2: 2.128309373571589, 3: 1.4229343002471617, 4: 1.05899442842631, 5: 0.5993519408000851}
Schedule Fitness:  -31.94246651840392
Subject 1:
	08:00 AM - 08:40 AM (40 minutes) (Break: 8 minutes)
	08:48 AM - 09:28 AM (40 minutes) (Break: 8 minutes)
	09:36 AM - 10:16 AM (40 minutes) (Break: 8 minutes)
	10:24 AM - 10:30 AM (6 minutes) (Break: 8 minutes)
Subject 2:
	10:38 AM - 11:18 AM (40 minutes) (Break: 8 minutes)
	11:26 AM - 12:06 PM (40 minutes) (Break: 8 minutes)
	12:14 PM - 12:35 PM (20 minutes) (Break: 8 minutes)
Subject 3:
	12:43 PM - 01:23 PM (40 minutes) (Break: 8 minutes)
	01:31 PM - 02:11 PM (40 minutes) (Break: 8 minutes)
	02:19 PM - 02:59 PM (40 minutes) (Break: 8 minutes)
	03:07 PM - 03:15 PM (7 minutes) (Break: 8 minutes)
Subject 4:
	03:23 PM - 04:03 PM (40 minutes) (Break: 8 minutes)
	04:11 PM - 04:51 PM (40 minutes) (Break: 8 minutes)
	04:59 PM - 05:04 PM (5 minutes) (Break: 8 minutes)
Subject 5:
	05:1

### Student 7

In [12]:
# Example parameters for the study schedule
num_subjects = 5  # Number of subjects
study_time = 8 # Total study time per day in hours (m hours)
priorities = {0: 2, 1: 3, 2: 5, 3: 1, 4: 4}  # Priority for each subject
breaktimes = {n:10/60 for n in range(0, num_subjects)}  # Breaks per subject in hours
start_time = "8:00 PM"  # Desired start time
num_days = 7

fitnesses = []
for day in range(num_days):
    print(f"Day {day + 1}:")
    PSOSA_optimize = PSOSA(
        num_subjects=num_subjects,
        study_time=study_time,
        priorities=priorities,
        breaktimes=breaktimes
    ).run()

    print("Generated Schedule: ", PSOSA_optimize.best_schedules)
    print("Schedule Fitness: ", PSOSA_optimize.best_fitness)
    fitnesses.append(PSOSA_optimize.best_fitness)
    format(result=PSOSA_optimize,start_time=start_time, breaktimes=breaktimes)

Day 1:
Generated Schedule:  {0: 1.154838574934699, 1: 1.4724372490903714, 2: 2.2864115949669137, 3: 1.095598891744357, 4: 1.9907136892636603}
Schedule Fitness:  -24.771101171173328
Subject 1:
	08:00 PM - 08:50 PM (50 minutes) (Break: 10 minutes)
	09:00 PM - 09:19 PM (19 minutes) (Break: 10 minutes)
Subject 2:
	09:29 PM - 10:19 PM (50 minutes) (Break: 10 minutes)
	10:29 PM - 11:07 PM (38 minutes) (Break: 10 minutes)
Subject 3:
	11:17 PM - 12:07 AM (50 minutes) (Break: 10 minutes)
	12:17 AM - 01:07 AM (50 minutes) (Break: 10 minutes)
	01:17 AM - 01:54 AM (37 minutes) (Break: 10 minutes)
Subject 4:
	02:04 AM - 02:54 AM (50 minutes) (Break: 10 minutes)
	03:04 AM - 03:20 AM (15 minutes) (Break: 10 minutes)
Subject 5:
	03:30 AM - 04:20 AM (50 minutes) (Break: 10 minutes)
	04:30 AM - 05:20 AM (50 minutes) (Break: 10 minutes)
	05:30 AM - 05:50 AM (19 minutes) (Break: 10 minutes)
Day 2:
Generated Schedule:  {0: 1.2475204528375514, 1: 1.640066633366718, 2: 2.086806710496322, 3: 1.05972816799595,